# HDS5210-2022 Midterm

In the midterm, you're going to use all of the programming and data management skills you've developed so far to build a risk calculator that pretends to be integrated with a clinical registry.  You'll compute the PRIEST COVID-19 Clinical Severity Score for a series of patients and, based on their risk of an adverse outcome, query a REST web service to find a hospital to transfer them to. The end result of your work will be a list of instructions on where each patient should be discharged: "home" if they are below 30% risk and the recommended hospital if they are at or above 30%.

Each step in the midterm will build up to form your final solution. Along the way, I've provided plenty of test cases to make sure that you're getting those steps correct.

The midterm is due at 11:59 PM CST on Monday, March 14th.

---

## Step 1: Calculate PRIEST Clinical Severity Score

This scoring algorithm can be found [here on the MDCalc website](https://www.mdcalc.com/priest-covid-19-clinical-severity-score#evidence).  

1. You will need to write a function called **priest()** with the following input parameters.  
 * Sex
 * Age in years
 * Respiratory rate in breaths per minute
 * Oxygen saturation as a percent between 0 and 1
 * Heart rate in beats per minute
 * Systolic BP in mmHg
 * Temperature in degrees C
 * Alertness as a string description
 * Inspired Oxygen as as string description
 * Performance Status as a string description
2. The function will need to follow the algorithm provided on the MDCalc website to compute a risk percentage that should be returned as a numeric value between 0 and 1.
3. Be sure to use docstring documentation and at least three built-in docstring test cases.
4. Assume that the input values that are strings could be any combination of upper or lower case. For example: 'male', 'Male', 'MALE', 'MalE' should all be interpretted by your code as male.
5. If any of the inputs are invalid (for example a sex value that is not recognizable as male or female) your code should raise a ValueException that includes a message with the invalid input and which parameter that input was provided as.

NOTES:
1. In the final step there is a rule to convert form raw score to percentile.  In that conversion, 17-25 maps to 59-88% and ≥26 maps to >99%.  For our purposes, we want these to be specific % number outputs.  Use the following rule:
 * If score is between 17 and 25, percentile should be 0.59
 * If score is greater than or equal to 26, percentile should be 0.99


In [1]:
def priest(sex, age, resp_rate, o2_saturate, hr, syst_bp, temp_c, alertness, insp_o2, perform_stat):
    '''
    This function takes 10 different parameters  provided on the MDCalc website to compute a risk percentage
    that is returned as a numeric value between 0 and 1.
    
    >>> print(priest('MALE', 80, 22.0, 0.96, 41.0, 115.0, 38.1, 'ALERT', 'AIR', 'limited activity, can self-care'))
    0.29
    >>> print(priest('FEMALE', 61, 23.0, 0.96, 105.0, 95.0, 34.9, 'ALERT', 'AIR', 'unrestricted normal activity'))
    0.29
    >>> print(priest('FEMALE', 66, 23.0, 0.96, 135.0, 89.0, 40.8, 'ALERT', 'supplemental oxygen', 'unrestricted normal activity'))
    0.49
       
    '''
    score = 0
    percentage = {0:1,1:1,2:2,3:2,4:3,5:9,6:15,7:18,8:22,9:26,10:29,11:34,12:38,13:46,14:47,15:49,16:55}
    result = 0.0
    perform_stat = {"unrestricted normal activity":0, "limited strenuous activity, can do light activity":1,"limited activity, can self-care":2, "limited self-care":3, "bed/chair bound, no self-care":4}
    sex = sex.lower()
       
    if sex == 'male':
        score += 1
    elif sex == 'female':
        score += 0        
    else:
        raise ValueError("Invalid Input. The sex can be either male or female")

    if age < 49:
        score += 0
    elif age <= 65 and age >= 50:
        score += 2
    elif age <= 80 and age >= 66:
        score += 3
    elif age > 80:
        score += 4
    else:
        raise ValueError("Invalid Input. age must be greater than 16")

    if resp_rate < 9:
        score += 3
    elif resp_rate <= 11 and resp_rate >= 9:
        score += 1
    elif resp_rate <= 20 and resp_rate >= 12:
        score += 0
    elif resp_rate > 24:
        score += 3
    else:
        raise ValueError("Invalid Input. please enter valid respiratory rate")
        
        
    if o2_saturate > 95:
        score += 0
    elif o2_saturate <= 95 and o2_saturate >= 95:
        score += 1
    elif o2_saturate <= 93 and o2_saturate >= 92:
        score += 2
    elif o2_saturate < 92:
        score += 3
    
    if hr < 41:
        score += 3
    elif hr <= 50 and hr >= 41:
        score += 1
    elif hr <= 90 and hr >= 51:
        score += 0
    elif hr <= 100 and hr >= 91:
        score += 1
    elif hr <= 130 and hr >= 111:
        score += 2
    elif hr > 130:
        score += 3
    else: 
        raise ValueError("Invalid Input. please enter valid heart rate")

    if syst_bp < 91:
        score += 3
    elif syst_bp <= 100 and syst_bp >= 91:
        score += 2
    elif syst_bp <= 110 and syst_bp >= 100:
        score += 1
    elif syst_bp <= 219 and syst_bp >= 111:
        score += 0
    elif syst_bp >= 219:
        score += 3
    else:
        raise ValueError("Invalid Input. please enter valid systolic bp")
        
    
    if temp_c < 35.1:
        score += 3
    elif temp_c <= 36 and temp_c >= 35.1:
        score += 1
    elif temp_c <= 38 and temp_c >= 36.1:
        score += 0
    elif temp_c <= 39 and temp_c >= 38.1:
        score += 1
    elif temp_c > 39:
        score += 2
    else:
        raise ValueError("Invalid Input. please enter valid temperature")
        
    if alertness == 'Alert':
        score += 0
    elif alertness == 'Confused or not alert':
        score += 3
    else:
        raise ValueError("Invalid Input. The alertness value can be either elert or confused or not alert")
        
        
    if insp_o2 == 'Air':
        score += 0
    elif insp_o2 == 'Supplemental oxygen':
        score += 2
    else:
        raise ValueError("Invalid Input. The inspired oxygen should be either Air or supplemental oxygen")
    
    if perform_stat == 'Unrestricted normal activity':
        score += 0
    elif perform_stat == 'Limited strenuous activity, can do light activity':
        score += 1
    elif perform_stat == 'Limited activity, can self-care':
        score += 2
    elif perform_stat == 'Limited self-care':
        score += 3
    elif perform_stat == 'Bed/chair bound, no self-care':
        score += 4
    else:
        raise ValueError("Invalid Input. Please give valid performance status")
    
    if priest_score <= 1:
        percent = 0.01
    elif priest_score <= 3 and priest_score >= 2:
        percent = 0.02
    elif priest_score == 4:
        percent = 0.03
    elif priest_score == 5:
        percent = 0.09
    elif priest_score == 6:
        percent = 0.15
    elif priest_score == 7:
        percent = 0.18
    elif priest_score == 8:
        percent = 0.22
    elif priest_score == 9:
        percent = 0.26
    elif priest_score == 10:
        percent = 0.29
    elif priest_score == 11:
        percent = 0.34
    elif priest_score == 12:
        percent = 0.38
    elif priest_score == 13:
        percent = 0.46
    elif priest_score == 14:
        percent = 0.47
    elif priest_score == 15:
        percent = 0.49
    elif priest_score == 16:
        percent = 0.55
    elif priest_score <= 25 and priest_score >= 17:
        percent = 0.59
    elif priest_score >= 25:
        percent = 0.99

        
    return percent


In [2]:
import doctest
doctest.run_docstring_examples(priest, globals(), verbose=True)

Finding tests in NoName
Trying:
    print(priest('MALE', 80, 22.0, 0.96, 41.0, 115.0, 38.1, 'ALERT', 'AIR', 'limited activity, can self-care'))
Expecting:
    0.29
**********************************************************************
File "__main__", line 6, in NoName
Failed example:
    print(priest('MALE', 80, 22.0, 0.96, 41.0, 115.0, 38.1, 'ALERT', 'AIR', 'limited activity, can self-care'))
Exception raised:
    Traceback (most recent call last):
      File "/opt/tljh/user/lib/python3.6/doctest.py", line 1330, in __run
        compileflags, 1), test.globs)
      File "<doctest NoName[0]>", line 1, in <module>
        print(priest('MALE', 80, 22.0, 0.96, 41.0, 115.0, 38.1, 'ALERT', 'AIR', 'limited activity, can self-care'))
      File "<ipython-input-1-b46f42136a91>", line 47, in priest
        raise ValueError("Invalid Input. please enter valid respiratory rate")
    ValueError: Invalid Input. please enter valid respiratory rate
Trying:
    print(priest('FEMALE', 61, 23.0, 0.96, 10

## Part 2: Find a hospital

The next thing we have to do is figure out where to send this particular patient.  The guidelines on where to send a patient are based on their age (pediatric, adult, geriatric), sex, and risk percentage.  Luckily, you don't have to implement these rules. I already have. All you have to do is use a REST web service that I've created for you.

You'll want to use Python to make a call to my REST web service similar to the example URL below. The first part of the URL will be the same for everyone and every request that you make. What you will need to modify for each of your requests is the information after the question mark.

```
https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/?age=40&sex=male&risk_pct=0.1
```

The example above asks my web service where a 40-year old male with a risk percetage of 10% should go.  What the web service will return back is a JSON string containing the information you need.  That JSON will look like this:

```json
{
  "age": "40",
  "sex": "male",
  "risk": "0.1",
  "hospital": "Southwest Hospital and Medical Center"
}
```

1. Your job is to write a function called **find_hospital()** that takes age, sex, and risk as parameters.
2. Your function should call this REST web service
3. Then your function will need to interpret the JSON it gets and return just the name of the hospital
4. If anything fails, return None without raising any exceptions
5. Include a good docstring with at least five test cases.


In [3]:
import requests
def find_hospital(age, sex, risk):
    
    '''
    This function takes age, sex and risk as parameters and outputs the name of the hospital using REST web service.
    
    >>> print(find_hospital(51,"male", 0.22))
    Southwest Hospital and Medical Center

    >>> print(find_hospital(85,"female",0.46)) 
    Wesley Woods Geriatric Hospital

    >>> print(find_hospital(65,"male", 0.18))
    Wesley Woods Geriatric Hospital

    >>> print(find_hospital(18,"female", 0.03))
    Select Specialty Hospital - Northeast Atlanta

    >>> print(find_hospital(18,"male", 0.26))
    Southwest Hospital and Medical Center
    
    '''
    try:
        page = "https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/?age="+str(age)+"&sex="+sex+"&risk_pct="+str(risk)
        response = requests.get(page)
        data = response.json()
        result = data['hospital']
        return(result)
    except:
        return(None)


    

In [4]:
import doctest
doctest.run_docstring_examples(find_hospital, globals(), verbose=True)

Finding tests in NoName
Trying:
    print(find_hospital(51,"male", 0.22))
Expecting:
    Southwest Hospital and Medical Center
ok
Trying:
    print(find_hospital(85,"female",0.46)) 
Expecting:
    Wesley Woods Geriatric Hospital
ok
Trying:
    print(find_hospital(65,"male", 0.18))
Expecting:
    Wesley Woods Geriatric Hospital
ok
Trying:
    print(find_hospital(18,"female", 0.03))
Expecting:
    Select Specialty Hospital - Northeast Atlanta
ok
Trying:
    print(find_hospital(18,"male", 0.26))
Expecting:
    Southwest Hospital and Medical Center
ok


## Part 3: Get the address for that hospital from a webpage

Great! Now we have code to tell us which hospital to send someone to... but we don't know where that hospital is. The next function we need to create is one that looks up the address of that hospital.  All of these hospitals are in Atlanta, Georgia.  We're going to use the list on this webpage to lookup the address for that hospital, based on its name.

https://www.officialusa.com/stateguides/health/hospitals/georgia.html

1. You need to create a function called **get_address()** that takes hospital name as a parameter and searches the data on the webpage above to find the addres for that hospital.
2. I've said that all the hospitals are in Atlanta, but this webpage has hospitals from all of Georgia.  So, make sure you verify that the row of data you're using is in Atlanta, just in case there are hospitals with the same name in different cities.
3. Your code will have to parse the HTML on the webpage and turn that into some kind of data structure that you can search through to find the right hospital.
4. If you do find more than one matching hospital in Atlanta with the same name, you should raise a KeyError.
5. If the hospital name isn't found, the function should raise a KeyError.
6. Be sure to use good docstring documentation and includes at least 3 test cases.

In [5]:
import requests
from bs4 import BeautifulSoup

def get_address(hospital):
    
    '''
    This function takes hospital name as parameter and returns its location.
    
    >>> print(get_address("Select Specialty Hospital - Northeast Atlanta"))
    1821 CLIFTON ROAD NE

    >>> print(get_address("Emory Dunwoody Medical Center"))
    4500 NORTH SHALLOWFORD ROAD

    >>> print(get_address("Childrens Healthcare of Atlanta at Scottish Rite"))
    1001 JOHNSON FERRY RD
    
    '''
    
    page = requests.get('https://www.officialusa.com/stateguides/health/hospitals/georgia.html').content
    data = BeautifulSoup(page, 'page.parser')
    table = data.find(id='myTable')
    rows = table.find_all('tr')
    
    data = []
    for row in rows:
        columns = row.find_all('td')
        columns = [ele.text.strip() for ele in columns]
        data.append([ele for ele in columns if ele])
    data.pop(0)
    struc = {}
    for info in data:
        if(info[0].lower() == "atlanta"):
            if info[1].lower() in struc:
                struc[info[1].lower()].append(info[3].lower())
            else:
                struc[info[1].lower()] = [info[3].lower()]
    
    if(hospital.lower() in struc):
        if(len(struc[hospital.lower()]) > 1):
            raise KeyError("multiple hospitals in Atlanta with the same names")
        else:
            return(struc[hospital.lower()][0])
    else:
        raise KeyError("Hospital not found")


In [6]:
import doctest
doctest.run_docstring_examples(find_hospital, globals(), verbose=True)

Finding tests in NoName
Trying:
    print(find_hospital(51,"male", 0.22))
Expecting:
    Southwest Hospital and Medical Center
ok
Trying:
    print(find_hospital(85,"female",0.46)) 
Expecting:
    Wesley Woods Geriatric Hospital
ok
Trying:
    print(find_hospital(65,"male", 0.18))
Expecting:
    Wesley Woods Geriatric Hospital
ok
Trying:
    print(find_hospital(18,"female", 0.03))
Expecting:
    Select Specialty Hospital - Northeast Atlanta
ok
Trying:
    print(find_hospital(18,"male", 0.26))
Expecting:
    Southwest Hospital and Medical Center
ok


## Part 4: Run the risk calculator on a population

The `/data` folder has a file called `people.psv`.  It is a pipe-delimited (`|`) file with columns that match the inputs for the PRIEST calculation above.

In addition, the file has a patient identifier in the first column.

1. Write a function called **process_people()** that takes a file name as a parameter. Your Python program should use your code above to process all of these rows, determine the hospital and address, and return a list whose items are a dictionary like `{ patient_number: [sex, age, breath, o2sat, heart, systolic, temp, alertness, inspired, status, hospital, address]}`
2. Be sure to use good docstrings, but you don't need any tests in your doc strings.  I've provided those for you below.


**NOTE** that when running your code for all the 100 records in the `people.psv` file, it may take a few minutes to complete.  You're making multiple calls to the internet for each record, so that can take a little while.


## Part 5: Checking your final results

The final step is to check your results.  You should be able to compare your results to the output in `people_results.json` in the `/data` folder.  Write some code to check your results.  This does not need to be a function.

---

## Check your work above

If you didn't get them all correct, take a few minutes to think through those that aren't correct.


## Submitting Your Work

In order to submit your work, you'll need to use the `git` command line program to **add** your homework file (this file) to your local repository, **commit** your changes to your local repository, and then **push** those changes up to github.com.  From there, I'll be able to **pull** the changes down and do my grading.  I'll provide some feedback, **commit** and **push** my comments back to you.  Next week, I'll show you how to **pull** down my comments.

To run through everything one last time and submit your work:
1. Use the `Kernel` -> `Restart Kernel and Run All Cells` menu option to run everything from top to bottom and stop here.
2. Follow the instruction on the prompt below to either ssave and submit your work, or continue working.

If anything fails along the way with this submission part of the process, let me know.  I'll help you troubleshoort.

---

In [ ]:
a=input('''
Are you ready to submit your work?
1. Click the Save icon (or do Ctrl-S / Cmd-S)
2. Type "yes" or "no" below
3. Press Enter

''')

if a=='yes':
    !git add "midterm-2022.ipynb"
    !git commit -a -m "Submitting the midterm"
    !git push
else:
    print('''
    
OK. We can wait.
''')